In [1]:
import random,math,timeit
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection as ms

In [2]:
def random_search(n,dim):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens



In [3]:
def case1(move):
    return 1 if random.uniform(-0.1,0.9)<move else 0
def case2(one_bin):
    if random.uniform(-0.1,0.9)<math.tanh(int(one_bin)):
        if one_bin==1:
            return 0
        else:return 1
    else:return 0
def exchange_binary(binary,score,alpha,beta,gamma):
    al_binary=binary
    e=0.05*random.uniform(0,1)
    movement=beta*math.exp(-gamma*score**2)*score + alpha*e
    if random.uniform(0,1) < movement:
        for i,b in enumerate(binary):
            move=beta*math.exp(-gamma*b**2)*b + alpha*e
            al_binary[i]=case1(move)
    else:
        for i,b in enumerate(binary):
            move=beta*math.exp(-gamma*b**2)*b + alpha*e
            al_binary[i]=case2(move)
    return al_binary

In [4]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= RandomForestClassifier(n_estimators=25).fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [5]:
def BFA(train_d,train_l,n=20,max_iter=25,gamma=0.20,beta=0.20,alpha=0.25):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    dim=len(train_d[0])
    global_best=float("-inf")
    global_position=tuple([0]*dim)
    gens_dict = {tuple([0]*dim):float("-inf")}
    gens=random_search(n,dim)
    for gen in gens:
        if tuple(gen) in gens_dict:
            score = gens_dict[tuple(gen)]
        else:
            score=evaluate(train_d,train_l,gen)
            gens_dict[tuple(gen)]=score
        if score > global_best:
            global_best=score
            global_position=dc(gen)
    for it in range(max_iter):
        for i,x in enumerate(gens):
            for j,y in enumerate(gens):
                if gens_dict[tuple(y)] < gens_dict[tuple(x)]:
                    gens[j]=exchange_binary(y,gens_dict[tuple(y)],alpha,beta,gamma)
                gen = gens[j]
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gens[j])
                    gens_dict[tuple(gen)]=score
                if score > global_best:
                    global_best=score
                    global_position=dc(gen)
    return global_position,global_position.count(1)


In [6]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([RandomForestClassifier(n_estimators=25).fit(al_data,tr_y).score(al_test_data,te_y) for i in range(5)])

In [7]:
with open("/home/shubbham28/Downloads/bc/wdbc1.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/bc/wdbc2.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

IOError: [Errno 2] No such file or directory: '/home/shubbham28/Downloads/bc/wdbc1.csv'

In [23]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BFA(train_d,train_l,n=20,max_iter=25,gamma=0.20,beta=0.20,alpha=0.25)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
final=np.argsort(flist)[::-1][:fattr]
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.9538461538461538
1  110010001000000000000010100000  6  0.948252
2  010000000100000000000010100100  5  0.942657
3  110100000010000000100100100100  8  0.960839
4  010010101000000000000010100010  7  0.955245
5  010000000000000011001000100001  6  0.952448
6  010000101000000110001000101101  10  0.963636
7  011111110111111111011111111111  27  0.959441
8  010000000001000000000001111000  6  0.953846
9  110000000000000000000010100000  4  0.952448
10  010000000000000100000001100000  4  0.955245
11  100010000010000001010110000100  8  0.949650
12  010100100000000000001000100000  5  0.956643
13  010000000000010000000010100000  4  0.949650
14  001000010000000000001110000000  5  0.956643
15  000110000000000000000110001011  7  0.948252
16  010000000100000100000011101101  9  0.944056
17  000000000000001100100110100000  6  0.955245
18  000000010001000100000101000100  6  0.958042
19  000000000000000001000101001100  5  0.962238
20  111111111111110111111110111111  28  0.953846
Final:   010000000000000100

In [8]:
with open("/home/shubbham28/Downloads/Arrhythmia/heart.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/Arrhythmia/heart_output.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [9]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BFA(train_d,train_l,n=20,max_iter=25,gamma=0.20,beta=0.20,alpha=0.25)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
print flist,fattr
final=np.argsort(flist)[::-1][:fattr]
print final
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.7529411764705882
1  1001001010111  7  0.720588
2  0010000000111  4  0.750000
3  0001000001011  4  0.679412
4  1001000101011  6  0.658824
5  0000000000111  3  0.676471
6  0011010100111  7  0.717647
7  1110110111111  11  0.729412
8  0111101111111  11  0.755882
9  0100000101111  6  0.673529
10  0000000010111  4  0.720588
11  0000000000111  3  0.676471
12  1111011111111  12  0.773529
13  0000000010111  4  0.720588
14  1111110111111  12  0.750000
15  0111111011011  10  0.767647
16  0011000000011  4  0.732353
17  1110101110111  10  0.758824
18  0000000000111  3  0.676471
19  0011010000111  6  0.711765
20  1111111001111  11  0.767647
[7, 8, 11, 11, 6, 7, 6, 8, 9, 9, 16, 20, 20] 6
[12 11 10  3  2  9]
Final:   0011000001111   6   0.720882    2379.1885
